# Project Kaggle

In this project, you will be using XGBoost to compete in the social influence competition: https://www.kaggle.com/c/predict-who-is-more-influential-in-a-social-network

We focus on this project since its data size is manageable and we can easily manipulate it. 

You need to:
1. Read and understand the competition objective
2. Download the data for this competition
3. Constructive an XGBoost model for this competition
4. Tune your XGBoost model
5. Submit your results


In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb

train = pd.read_csv("./all/train.csv")
test = pd.read_csv("./all/test.csv")
train.head()

,Choice,A_follower_count,A_following_count,A_listed_count,A_mentions_received,A_retweets_received,A_mentions_sent,A_retweets_sent,A_posts,A_network_feature_1,...,B_following_count,B_listed_count,B_mentions_received,B_retweets_received,B_mentions_sent,B_retweets_sent,B_posts,B_network_feature_1,B_network_feature_2,B_network_feature_3
0,0,228,302,3,0.583979,0.100503,0.100503,0.100503,0.362150,2,...,29808,1689,15.430498,3.984029,8.204331,0.332423,6.988815,66,75.530303,1916.893939
1,0,21591,1179,228,90.456506,25.798292,5.709329,1.111159,5.176620,369,...,848,1610,40.495021,8.943607,3.227677,0.564343,1.070321,163,132.030675,2931.515337
2,0,7310,1215,101,25.503644,9.556347,5.361519,0.591206,3.589718,95,...,482,206,0.734696,0.354379,0.603202,0.100503,0.603202,3,10.333333,277.333333
3,0,20,7,2,7.690824,0.277306,1.331508,0.100503,2.830627,6,...,17637,278,572.874856,390.293681,27.552040,7.167557,32.101906,1762,23.469296,1395.845634
4,1,45589,862,2641,148.854279,36.998884,27.881768,3.333492,23.861282,551,...,1711,181,21.601866,3.581661,6.764657,1.119727,4.563246,85,48.500000,1993.627907


In [ ]:
# Feature engineering
# Compute the following features:
#   - followers/following
#   - mentions received/sent
#   - retweets received/sent
#   - followers/posts
#   - retweets received/posts
#   - mentions received/posts


In [14]:
X = train.loc[:, "A_follower_count":]
y = train.loc[:,"Choice"]
data_dmatrix = xgb.DMatrix(data=X,label=y)
params = {"objective":"binary:logistic",'eval_metric': 'auc', 
          "eta":0.005, "gamma":1, "lambda":3, "max_depth":4, "min_child_weight":1,
          "verbose":False, "subsample":0.8, "colsample_bytree":0.8, 'silent': 1}
cv_result = xgb.cv(dtrain = data_dmatrix, nfold = 5, params=params,num_boost_round=3000)

In [16]:
model = xgb.train(dtrain = data_dmatrix, params=params,num_boost_round=3000)

In [19]:
preds = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
df_submission = pd.DataFrame({"Id":range(1,len(preds)+1), "Choice":preds})
df_submission.head()

,Id,Choice
0,1,0.198136
1,2,0.764119
2,3,0.036787
3,4,0.181995
4,5,0.587838


In [22]:
df_submission.to_csv("~/Downloads/submission.csv", index=False)

In [21]:
preds

array([0.19813634, 0.76411915, 0.03678663, ..., 0.6418845 , 0.5343805 ,
       0.2428439 ], dtype=float32)